In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

In [2]:
demand_data = pd.read_csv('data\dataset\demand.csv')
vehicle_data = pd.read_csv('vehicle_data_edit.csv')
vehicle_fuels = pd.read_csv('data/dataset/vehicles_fuels.csv')
fuels_data = pd.read_csv('data/dataset/fuels.csv')
carbon_emissions_data = pd.read_csv('data/dataset/carbon_emissions.csv')
cost_profiles = pd.read_csv('data/dataset/cost_profiles.csv')

In [3]:
avail_vehicle = {'ID': [],
                 'buy_cost': [],
                 'distance': []}

In [4]:
total_fleet = {'ID': [],
               'buy_cost': []}

In [5]:

buy = {}
use = {}
sell = {}

In [6]:
def feasible_vehicle(yr, s,d):
    temp_df = vehicle_data.loc[(vehicle_data['Year'] == yr) & (vehicle_data['Size'] == s)]
    if d == 'D2':
        temp_df = temp_df.loc[temp_df['Distance'] != 'D1']
    elif d == 'D3':
        temp_df = temp_df.loc[(temp_df['Distance'] == 'D3') | (temp_df['Distance'] == 'D4')]
    elif d == 'D4':
        temp_df = temp_df.loc[(temp_df['Distance'] == 'D4')]
    return temp_df

def decide_vehicle(feas_veh, prob):
    rand = random.random()
    for i in range(len(prob)):
        if rand < sum(prob[:i+1]):
            return feas_veh.iloc[i]
        
def feasible_from_avail_vehicle(avail_vehicle, feas_veh):
    new_feas_vehicle = []
    arr_distance = []
    for k in range(len(avail_vehicle['ID'])):
        i = avail_vehicle['ID'][k]
        if i in feas_veh['ID'].tolist():
            new_feas_vehicle.append(i)
            arr_distance.append(avail_vehicle['distance'][k])
    return {'ID': new_feas_vehicle, 'distance': arr_distance}

def sort_vehicle(vehicle_dict):
    list_BEV = []
    list_BEV_dist = []
    list_LNG = []
    list_LNG_dist = []
    list_diesel = []
    list_diesel_dist = []
    n = len(vehicle_dict['ID'])
    for k in range(n):
        i = vehicle_dict['ID'][0]
        temp = i.split('_')
        if temp[0] == 'BEV':
            list_BEV.append(i)
            list_BEV_dist.append(vehicle_dict['distance'][0])
        elif temp[0] == 'LNG':
            list_LNG.append(i)
            list_LNG_dist.append(vehicle_dict['distance'][0])
        else:
            list_diesel.append(i)
            list_diesel_dist.append(vehicle_dict['distance'][0])
        del vehicle_dict['ID'][0]
        del vehicle_dict['distance'][0]

    return {'ID': list_BEV + list_LNG + list_diesel, 'distance': list_BEV_dist + list_LNG_dist + list_diesel_dist}

# def carbon_capture():

In [7]:
unique_year = np.unique(demand_data['Year'])
s = np.unique(demand_data['Size'])
d = np.unique(demand_data['Distance'])

In [8]:
#initial
year = 2023
init_demand = demand_data.loc[demand_data['Year'] == year]
buy[year] = {'ID': [],
             'cost': [],
             'distance': []}
demand_avail_dict = {}
demand_dict = {}
for i in range(len(s)):
    demand_dict[s[i]] = {}
    demand_avail_dict[s[i]] = {}
    demand_avail = 0
    for j in range(len(d)-1,-1,-1):
        demand = init_demand['Demand (km)'].loc[(init_demand['Size'] == s[i]) & (init_demand['Distance'] == d[j])].iloc[0]
        demand_dict[s[i]][d[j]] = demand
        feas_veh = feasible_vehicle(year,s[i],d[j])
        yearly_cost = 0.12*feas_veh['Cost ($)'] + feas_veh['fuel_cost']
        feas_veh['yearly_cost'] = yearly_cost
        inv = 1/feas_veh['yearly_cost']
        inv[inv.idxmax()] = inv[inv.idxmax()]*6
        prob = (inv/sum(inv)).values
        while demand_avail < demand:
            temp_df_decide = decide_vehicle(feas_veh, prob)
            buy[year]['ID'].append(temp_df_decide['ID'])
            buy[year]['distance'].append(temp_df_decide['Yearly range (km)'])
            buy[year]['cost'].append(temp_df_decide['Cost ($)'])
            demand_avail = demand_avail + temp_df_decide['Yearly range (km)']
        demand_avail_dict[s[i]][d[j]] = demand_avail
        demand_avail = demand_avail - demand

C:\Users\asus\AppData\Local\Temp\ipykernel_3336\2136716154.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feas_veh['yearly_cost'] = yearly_cost
C:\Users\asus\AppData\Local\Temp\ipykernel_3336\2136716154.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feas_veh['yearly_cost'] = yearly_cost
C:\Users\asus\AppData\Local\Temp\ipykernel_3336\2136716154.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [9]:
buy[year]['cost']

[100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 85000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 187000,
 100000,
 187000,
 100000,
 100000,
 100000,
 100000,
 133000,
 133000,
 133000,
 133000,
 133000,
 133000,
 133000,
 133000,
 133000,
 133000,
 104000,
 133000,
 133000,
 133000,
 133000,
 133000,
 133000,
 133000,
 133000,
 104000,
 133000,
 133000,
 133000,
 133000,
 133000,
 133000,
 133000,
 133000,
 133000,
 133000,
 133000,
 133000,
 151136,
 151136,
 151136,
 151136,
 151136,
 151136,
 151136,
 151136,
 1

In [10]:
avail_vehicle['ID'] = avail_vehicle['ID'] + buy[2023]['ID']
avail_vehicle['buy_cost'] = avail_vehicle['buy_cost'] + buy[year]['cost']
avail_vehicle['distance'] = avail_vehicle['distance'] + buy[2023]['distance']

In [11]:
total_fleet['ID'] = total_fleet['ID'] + buy[year]['ID']
total_fleet['cost_buy'] = total_fleet['buy_cost'] + buy[year]['cost']

In [12]:
use[2023] = {'ID': [],
             'size': [],
             'distance_bucket': [],
             'distance': []
             }
for i in range(len(s)):
    for j in range(len(d)-1,-1,-1):
        demand = init_demand['Demand (km)'].loc[(init_demand['Size'] == s[i]) & (init_demand['Distance'] == d[j])].iloc[0]
        feas_veh = feasible_vehicle(year,s[i],d[j])
        demand_satisfied = 0
        while demand - demand_satisfied != 0:
            new_feas_vehicle = feasible_from_avail_vehicle(avail_vehicle,feas_veh)
            sorted_feas_vehicle = sort_vehicle(new_feas_vehicle)
            veh = sorted_feas_vehicle['ID'][0]
            temp_df = feas_veh.loc[feas_veh['ID'] == veh]
            dist = min([demand - demand_satisfied,sorted_feas_vehicle['distance'][0]])
            demand_satisfied = demand_satisfied + dist
            use[year]['ID'].append(veh)
            use[year]['size'].append(s[i])
            use[year]['distance_bucket'].append(d[j])
            use[year]['distance'].append(dist)

            idx = avail_vehicle['ID'].index(veh)
            temp_list = [0 for i in range(len(avail_vehicle['distance']))]
            temp_list[idx] = dist

            avail_vehicle['distance'] = np.array(avail_vehicle['distance']) - np.array(temp_list)
            is_zero = np.argwhere(abs(avail_vehicle['distance']) <= 0.01).tolist()
            if len(is_zero) == 0:
                avail_vehicle['distance'] = avail_vehicle['distance'].tolist()
            else:
                avail_vehicle['distance'] = avail_vehicle['distance'].tolist()
                del avail_vehicle['ID'][is_zero[0][0]]
                del avail_vehicle['distance'][is_zero[0][0]]

In [13]:
sorted_feas_vehicle

{'ID': ['LNG_S4_2023'], 'distance': [68575]}

In [14]:
df_use = pd.DataFrame(use[2023])

In [15]:
#hitung carbon capture
cc = 0
for i in range(len(df_use)):
    veh = df_use['ID'].iloc[i]
    dist = df_use['distance'].iloc[i]
    temp_df = vehicle_fuels.loc[vehicle_fuels['ID'] == veh]
    veh_type = veh.split('_')
    veh_type = veh_type[0]
    if veh_type == 'BEV':
        fuel = 'Electricity'
    elif veh_type == 'LNG':
        fuel = 'LNG'
    else:
        fuel = 'B20'
    use[year]['fuel'] = fuel
    consumption_unit = temp_df.loc[(temp_df['Fuel'] == fuel)]
    consumption_unit = consumption_unit['Consumption (unit_fuel/km)'].iloc[0]
    emission = fuels_data.loc[(fuels_data['Fuel'] == fuel) & (fuels_data['Year'] == year)]
    emission = emission['Emissions (CO2/unit_fuel)'].iloc[0]
    cc_veh = consumption_unit*emission*dist
    cc = cc + cc_veh
cc

7179540.315979344

In [16]:
#hitung cost
#buy
cost_buy = 0
for i in buy[year]['ID']:
    veh_cost = vehicle_data.loc[(vehicle_data['ID'] == i)]
    veh_cost = veh_cost['Cost ($)'].iloc[0]
    cost_buy = cost_buy + veh_cost

#avail
cost_ins = 0
cost_mnt = 0
for i in range(len(avail_vehicle['ID'])):
    veh = avail_vehicle['ID'][i]
    year_buy = veh.split('_')
    year_buy = int(year_buy[-1])
    delta_year = year - year_buy + 1
    temp_df = cost_profiles.loc[cost_profiles['End of Year'] == delta_year]
    pct_ins = temp_df['Insurance Cost %'].iloc[0]/100
    pct_mnt = temp_df['Maintenance Cost %'].iloc[0]/100
    cost_ins = cost_ins + pct_ins*avail_vehicle['buy_cost'][i]
    cost_mnt = cost_mnt + pct_mnt*avail_vehicle['buy_cost'][i]

#cost fuel
use[year]['fuel'] = []
total_cost_fuel = 0
for i in range(len(use[year]['ID'])):
    veh = use[year]['ID'][i]
    dist = use[year]['distance'][i]
    temp_df = vehicle_fuels.loc[vehicle_fuels['ID'] == veh]
    veh_type = veh.split('_')
    veh_type = veh_type[0]
    if veh_type == 'BEV':
        fuel = 'Electricity'
    elif veh_type == 'LNG':
        fuel = 'LNG'
    else:
        fuel = 'B20'
    use[year]['fuel'].append(fuel)
    consumption_unit = temp_df.loc[(temp_df['Fuel'] == fuel)]
    consumption_unit = consumption_unit['Consumption (unit_fuel/km)'].iloc[0]
    cost_per_fuel = fuels_data.loc[(fuels_data['Fuel'] == fuel) & (fuels_data['Year'] == year)]
    cost_per_fuel = cost_per_fuel['Cost ($/unit_fuel)'].iloc[0]
    total_cost_fuel = total_cost_fuel + consumption_unit*cost_per_fuel*dist

total_cost = cost_buy + cost_ins + cost_mnt + total_cost_fuel
total_cost


28154214.345794123

In [17]:
max_sold = int(0.2*len(total_fleet['ID']))
max_sold

38

In [18]:
df_total_fleet = pd.DataFrame({'ID': total_fleet['ID'],
                               'cost_buy': total_fleet['cost_buy']})
df_total_fleet

,ID,cost_buy
0,LNG_S1_2023,100000
1,LNG_S1_2023,100000
2,LNG_S1_2023,100000
3,LNG_S1_2023,100000
4,LNG_S1_2023,100000
...,...,...
186,LNG_S4_2023,193454
187,LNG_S4_2023,193454
188,LNG_S4_2023,193454
189,LNG_S4_2023,193454


In [19]:
next_ins = []
next_mnt = []
for i in range(len(df_total_fleet)):
    veh = df_total_fleet['ID'].iloc[i]
    year_buy = veh.split('_')
    year_buy = int(year_buy[-1])
    delta_year = year - year_buy + 1
    temp_df = cost_profiles.loc[cost_profiles['End of Year'] == delta_year]
    pct_ins = temp_df['Insurance Cost %'].iloc[0]/100
    pct_mnt = temp_df['Maintenance Cost %'].iloc[0]/100
    next_ins.append(pct_ins*df_total_fleet['cost_buy'].iloc[i])
    next_mnt.append(pct_mnt*df_total_fleet['cost_buy'].iloc[i])

df_total_fleet['next_ins'] = np.array(next_ins)
df_total_fleet['next_mnt'] = np.array(next_mnt)
df_total_fleet['next_cost'] = df_total_fleet['next_ins'] + df_total_fleet['next_mnt']

In [20]:
df_total_fleet

,ID,cost_buy,next_ins,next_mnt,next_cost
0,LNG_S1_2023,100000,5000.0,1000.00,6000.00
1,LNG_S1_2023,100000,5000.0,1000.00,6000.00
2,LNG_S1_2023,100000,5000.0,1000.00,6000.00
3,LNG_S1_2023,100000,5000.0,1000.00,6000.00
4,LNG_S1_2023,100000,5000.0,1000.00,6000.00
...,...,...,...,...,...
186,LNG_S4_2023,193454,9672.7,1934.54,11607.24
187,LNG_S4_2023,193454,9672.7,1934.54,11607.24
188,LNG_S4_2023,193454,9672.7,1934.54,11607.24
189,LNG_S4_2023,193454,9672.7,1934.54,11607.24


In [21]:
sorted_df_total_fleet = df_total_fleet.sort_values(by=['next_cost'], ascending=False)
sorted_df_total_fleet

,ID,cost_buy,next_ins,next_mnt,next_cost
176,BEV_S3_2023,309090,15454.5,3090.90,18545.40
164,BEV_S3_2023,309090,15454.5,3090.90,18545.40
169,BEV_S3_2023,309090,15454.5,3090.90,18545.40
190,LNG_S4_2023,193454,9672.7,1934.54,11607.24
189,LNG_S4_2023,193454,9672.7,1934.54,11607.24
...,...,...,...,...,...
48,LNG_S1_2023,100000,5000.0,1000.00,6000.00
49,LNG_S1_2023,100000,5000.0,1000.00,6000.00
50,LNG_S1_2023,100000,5000.0,1000.00,6000.00
0,LNG_S1_2023,100000,5000.0,1000.00,6000.00


In [22]:
sell[year] = {'ID': [],
              'cost_sell': []}

sell[year]['ID'] = sorted_df_total_fleet['ID'].iloc[:max_sold].tolist()
arr_cost_sell = []
for i in range(len(sell[year]['ID'])):
    veh = df_total_fleet['ID'].iloc[i]
    year_buy = veh.split('_')
    year_buy = int(year_buy[-1])
    delta_year = year - year_buy + 1
    temp_df = cost_profiles.loc[cost_profiles['End of Year'] == delta_year]
    pct_resale = temp_df['Resale Value %'].iloc[0]/100
    arr_cost_sell.append(sorted_df_total_fleet['cost_buy'].iloc[i]*pct_resale)

sell[year]['cost_sell'] = arr_cost_sell

In [23]:
#submission
arr_year = []
arr_ID = []
arr_num_vehicle = []
arr_type = []
arr_fuel = []
arr_distance_bucket = []
arr_distance_per_vehicle = []

#buy
values_ID, counts = np.unique(buy[year]['ID'], return_counts=True)
for i in range(len(values_ID)):
    arr_year.append(year)
    arr_ID.append(values_ID[i])
    arr_num_vehicle.append(counts[i])
    arr_type.append('Buy')
    arr_fuel.append(None)
    arr_distance_bucket.append(None)
    arr_distance_per_vehicle.append(0)

#use
df_use = pd.DataFrame(use[year])
for i in range(len(d)):
    temp_df0 = df_use.loc[(df_use['distance_bucket'] == d[i])]
    unique_fuel = np.unique(temp_df0['fuel'])
    for j in range(len(unique_fuel)):
        temp_df1 = temp_df0.loc[(temp_df0['fuel'] == unique_fuel[j])]
        unique_dist = np.unique(temp_df1['distance'])
        for k in range(len(unique_dist)):
            temp_df2 = temp_df1.loc[(temp_df1['distance'] == unique_dist[k])]
            values_ID, counts = np.unique(temp_df2['ID'], return_counts=True)
            for l in range(len(values_ID)):
                arr_year.append(year)
                arr_ID.append(values_ID[l])
                arr_num_vehicle.append(counts[l])
                arr_type.append('Use')
                arr_fuel.append(unique_fuel[j])
                arr_distance_bucket.append(d[i])
                arr_distance_per_vehicle.append(unique_dist[k])

#sell
values_ID, counts = np.unique(sell[year]['ID'], return_counts=True)
for i in range(len(values_ID)):
    arr_year.append(year)
    arr_ID.append(values_ID[i])
    arr_num_vehicle.append(counts[i])
    arr_type.append('Sell')
    arr_fuel.append(None)
    arr_distance_bucket.append(None)
    arr_distance_per_vehicle.append(0)

In [24]:
unique_dist

array([  1809,   6315,  27677,  59426,  73000, 102000, 106000],
      dtype=int64)

In [25]:
df_submission = pd.DataFrame({'Year': arr_year,
                              'ID': arr_ID,
                              'Num_Vehicles': arr_num_vehicle,
                              'Type': arr_type,
                              'Fuel': arr_fuel,
                              'Distance_bucket': arr_distance_bucket,
                              'Distance_per_vehicle(km)': arr_distance_per_vehicle})
df_submission

,Year,ID,Num_Vehicles,Type,Fuel,Distance_bucket,Distance_per_vehicle(km)
0,2023,BEV_S1_2023,2,Buy,None,None,0
1,2023,BEV_S3_2023,3,Buy,None,None,0
2,2023,Diesel_S1_2023,1,Buy,None,None,0
3,2023,Diesel_S2_2023,2,Buy,None,None,0
4,2023,Diesel_S3_2023,13,Buy,None,None,0
5,2023,LNG_S1_2023,68,Buy,None,None,0
6,2023,LNG_S2_2023,30,Buy,None,None,0
7,2023,LNG_S3_2023,64,Buy,None,None,0
8,2023,LNG_S4_2023,8,Buy,None,None,0
9,2023,Diesel_S2_2023,1,Use,B20,D1,4575


In [26]:
df_submission = df_submission.drop([2])

In [27]:
df_submission

,Year,ID,Num_Vehicles,Type,Fuel,Distance_bucket,Distance_per_vehicle(km)
0,2023,BEV_S1_2023,2,Buy,None,None,0
1,2023,BEV_S3_2023,3,Buy,None,None,0
3,2023,Diesel_S2_2023,2,Buy,None,None,0
4,2023,Diesel_S3_2023,13,Buy,None,None,0
5,2023,LNG_S1_2023,68,Buy,None,None,0
6,2023,LNG_S2_2023,30,Buy,None,None,0
7,2023,LNG_S3_2023,64,Buy,None,None,0
8,2023,LNG_S4_2023,8,Buy,None,None,0
9,2023,Diesel_S2_2023,1,Use,B20,D1,4575
10,2023,Diesel_S1_2023,1,Use,B20,D1,32601


In [28]:
df_total_fleet = pd.DataFrame({'ID': total_fleet['ID'],
                               'cost_buy': total_fleet['cost_buy']})
df_total_fleet

,ID,cost_buy
0,LNG_S1_2023,100000
1,LNG_S1_2023,100000
2,LNG_S1_2023,100000
3,LNG_S1_2023,100000
4,LNG_S1_2023,100000
...,...,...
186,LNG_S4_2023,193454
187,LNG_S4_2023,193454
188,LNG_S4_2023,193454
189,LNG_S4_2023,193454


In [29]:
temp_dict = {'ID': 'tes',
             'cost_buy': 10000}
df_total_fleet.loc[len(df_total_fleet)] = temp_dict
# df_total_fleet = df_total_fleet.append(temp_dict, ignore_index = True)

In [30]:
df_total_fleet

,ID,cost_buy
0,LNG_S1_2023,100000
1,LNG_S1_2023,100000
2,LNG_S1_2023,100000
3,LNG_S1_2023,100000
4,LNG_S1_2023,100000
...,...,...
187,LNG_S4_2023,193454
188,LNG_S4_2023,193454
189,LNG_S4_2023,193454
190,LNG_S4_2023,193454


In [31]:
exm = pd.DataFrame({'a': [1,2,3,3,5]})
np.argmin(np.abs(exm['a'] - 3))

2

In [32]:
a = [1,2,3,4,5]
del a[4]

In [33]:
a

[1, 2, 3, 4]